This notebook provides Python code to perform a pipetting protocol to synthesized AuNP particles using the procedure described in the manuscript.

In [1]:
from opentrons.simulate import get_protocol_api

import json 
import pandas as pd 
from datetime import datetime
from pytz import timezone 
Pacific = timezone('US/Pacific')
from IPython.display import clear_output
import time

In [2]:
START_WELL = "F1"
volume_df = pd.read_csv('./opentrons/volumes_2.csv')
volume_df

,CTAB,Water,GCH,AA,SN,GS
0,96,115,59,1,5,24
1,96,66,59,23,32,24
2,96,84,59,6,31,24
3,96,85,59,24,12,24
4,96,90,59,27,4,24
5,96,88,59,29,4,24


In [3]:
protocol = get_protocol_api('2.18')
CUSTOM_LABWARE_PATH = "../opentrons_custom_labware/"
protocol.home()

/Users/pozzolabadmin/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pozzolabadmin/.opentrons/deck_calibration.json not found. Loading defaults


In [4]:
protocol.set_rail_lights(True)

In [5]:
# load all the labware modules

# Load 300 µL pipette and tiprack
tiprack_300 = protocol.load_labware(
    load_name="opentrons_96_tiprack_300ul",
    location=7)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2",
    mount="right",
    tip_racks=[tiprack_300]
    )

# Load 20 µL pipette and tiprack
tiprack_20 = protocol.load_labware(
    load_name="opentrons_96_tiprack_20ul",
    location=4)

p20 = protocol.load_instrument(
    instrument_name="p20_single_gen2",
    mount="left",
    tip_racks=[tiprack_20]
    )

# Load a custom holder for stock solutions
with open(CUSTOM_LABWARE_PATH+'20mlscintillation_12_wellplate_18000ul.json') as labware_file:
    stocks_def = json.load(labware_file)
    stocks = protocol.load_labware_from_definition(stocks_def, location=1)

# Load temperature module
# The temperature module is used to keep the growth solution at 30 degree celsius
# The well plate is loaded on top of a alumnium adapter 
temp_module = protocol.load_module(module_name="temperature module gen2", location=6)
alplate_adapter = temp_module.load_adapter("opentrons_aluminum_flat_bottom_plate")
plate = alplate_adapter.load_labware('corning_96_wellplate_360ul_flat')
temp_module.set_temperature(celsius=30)

In [6]:
# Offset modules based on manual insepection
tiprack_300.set_offset(x=0.50, y=1.10, z=0.00)
tiprack_20.set_offset(x=0.00, y=0.90, z=0.00)
plate.set_offset(x=0.00, y=1.50, z=0.00)

In [7]:
tiprack_20_wells = [well for row in tiprack_20.rows() for well in row]
tiprack_300_wells = [well for row in tiprack_300.rows() for well in row]
stocks_wells = [well for row in stocks.rows() for well in row]
plate_wells = [well for row in plate.rows() for well in row]

In [8]:
START_WELL_INDEX = next((i for i, well in enumerate(plate_wells) if well.well_name == START_WELL), None)
if START_WELL_INDEX is not None:
    print(f"The index of well {START_WELL} is {START_WELL_INDEX}.")
else:
    print(f"Well {START_WELL} not found.")

The index of well F1 is 60.


In [9]:
def synthesize(stock_index, ds):
    """ Synthesize AuNP by mixing components

    stock_index : index of the stock to add (int)
    ds : a pandas dataseries with volumes to be added. 

    Each pipetting step involves selecting a relevant pipette based on the volume,
    using it to aspirate from the source well (stock) and dispensing it in a target location
    on the well plate.

    """
    p20.pick_up_tip(tiprack_20_wells[int(stock_index)])
    p300.pick_up_tip(tiprack_300_wells[int(stock_index)])
    has_used_p20, has_used_p300 = False, False
    for index, value in ds.items():
        if value<20:
            pipette = p20 
            has_used_p20 = True
        else:
            pipette = p300
            has_used_p300 = True
        source_well = stocks_wells[int(stock_index)]
        target_well = plate_wells[int(index) + START_WELL_INDEX]
        print("Dispensing %s of %d from %s into well %s "%(ds.name, 
                                                           value, 
                                                           source_well.well_name, 
                                                           target_well.well_name)
                                                           ,end='\r', 
                                                           flush=False
                                                           )
        pipette.aspirate(value, source_well)
        pipette.dispense(value, target_well.top())
        if not stock_index==0: # only do this for non-CTAB components
            pipette.blow_out()

    if has_used_p20:
        p20.drop_tip()
    else:
        p20.return_tip()

    if has_used_p300:
        p300.drop_tip()
    else:
        p300.return_tip()

In [10]:
for stock_index,(_, stock_vol_series)  in enumerate(volume_df.items()):
    synthesize(stock_index, stock_vol_series)

Dispensing CTAB of 96 from A1 into well F1 
Dispensing CTAB of 96 from A1 into well F2 
Dispensing CTAB of 96 from A1 into well F3 
Dispensing CTAB of 96 from A1 into well F4 
Dispensing CTAB of 96 from A1 into well F5 
Dispensing CTAB of 96 from A1 into well F6 
Dispensing Water of 115 from A2 into well F1 
Dispensing Water of 66 from A2 into well F2 
Dispensing Water of 84 from A2 into well F3 
Dispensing Water of 85 from A2 into well F4 
Dispensing Water of 90 from A2 into well F5 
Dispensing Water of 88 from A2 into well F6 
Dispensing GCH of 59 from A3 into well F1 
Dispensing GCH of 59 from A3 into well F2 
Dispensing GCH of 59 from A3 into well F3 
Dispensing GCH of 59 from A3 into well F4 
Dispensing GCH of 59 from A3 into well F5 
Dispensing GCH of 59 from A3 into well F6 
Dispensing AA of 1 from A4 into well F1 
Dispensing AA of 23 from A4 into well F2 
Dispensing AA of 6 from A4 into well F3 
Dispensing AA of 24 from A4 into well F4 
Dispensing AA of 27 from A4 into well F5 

In [11]:
current_date_time = datetime.now(Pacific)
print(current_date_time.strftime('End Time: %H:%M:%S'))

End Time: 15:30:11


In [12]:
protocol.set_rail_lights(False)

In [13]:
protocol.home()

In [14]:
print("Keeping the temperature module on for 30 mins at the initial temperature")

Keeping the temperature module on for 30 mins at the initial temperature


In [15]:
def countdown(minutes):
    total_seconds = minutes * 60
    for remaining in range(total_seconds, 0, -1):
        # Convert seconds to minutes and seconds
        mins, secs = divmod(remaining, 60)
        # Clear output in the notebook for a clean update
        clear_output(wait=True)
        print(f"Time remaining: {mins:02d}:{secs:02d}")
        time.sleep(1)
    clear_output(wait=True)
    print("Time's up!")

# Start a 30-minute countdown
countdown(1)


Time's up!
